<a href="https://colab.research.google.com/github/AraiKensuke/AIiRPS/blob/master/Launch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import importlib

if importlib.util.find_spec("AIiRPS") is None:
  !git clone https://github.com/AraiKensuke/AIiRPS.git
if importlib.util.find_spec("pyPG") is None:
  ###  First, clone the pyPG package from Github.  This will only last for this runtime session.  Do this every time you re-open notebook.
  !git clone https://github.com/AraiKensuke/pyPG.git
  %cd /content/pyPG/

  ###  pyPG is written in C++/python, and uses the C library GNU Scientific Library.  
  !sudo apt-get install libgsl-dev   

  ###  setup and build pyPG
  !python setup.py build_ext --inplace

  %cd /content   

if importlib.util.find_spec("LOSTtmp") is None:
  ###  LOST  (FFBS algorithm implemented here)
  !git clone https://github.com/AraiKensuke/LOSTtmp.git

  %cd /content/LOSTtmp/
  !python setup.py build_ext --inplace
  %cd /content

sys.path.insert(1, "/content/pyPG")    #  add these to searchpath for python modules
sys.path.insert(1, "/content/LOSTtmp") #  add these to searchpath for python modules
